## Park_HeeJI_HW03.ipynb
- Student Name: Hee Ji Park
- Github username : heejipark 
- USC ID: 4090715830
- Contact Email: heejipar@usc.edu

## Time Series Classification

In this problem, I will classify the activities of humans based on time series obtained by a Wireless Sensor Network.

The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity. Each file containis 6 time series collected from activities of the same person, which are called avgrss12, varrss12, avgrss13, varrss13, vgrss23, and arrss23. There are 88 instances in the dataset, each of which contains 6 time series and each time series has 480 consecutive values.

### Required libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import bootstrap
import numpy as np
import os

### (b) Set train and test data
- Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

In [2]:
dirFile = './../data/AReM/'
dirList = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
colName = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
dataList = ['dataset1.csv', 'dataset2.csv', 'dataset3.csv']

train = pd.DataFrame(columns=colName)
test = pd.DataFrame(columns=colName)

for dname in dirList:
    file = dirFile + dname 
    fileList = os.listdir(file+'/')
    
    for csv in fileList:
        df = pd.read_csv(file+'/'+csv, skiprows=4)
        df['FileName'] = dname
        i = int(csv[7])
        if csv[8] != '.': 
            i = int(csv[7:9])
        
        if df.isnull().values.any():
            print("Different format in excel file : ", dname,'/',csv)
            for i in range(len(df)):
                val = df.loc[i, '# Columns: time'].split()
                df.loc[i, '# Columns: time'] = val[0]
                for icol, col in enumerate(colName):
                    df.loc[i, col] = val[icol+1]
            print("Format Change Completed: ", dname,'/',csv)
        
        
        if 'bending' in dname:
            if i > 2: # train
                train = pd.concat([train, df], ignore_index=True)
            if i <= 2: # test
                test = pd.concat([test, df], ignore_index=True)    
        else:
            if i > 3: # train
                train = pd.concat([train, df], ignore_index=True)
            if i <= 3:  # test
                test = pd.concat([test, df], ignore_index=True)

Different format in excel file :  bending2 / dataset4.csv
Format Change Completed:  bending2 / dataset4.csv


In [3]:
print(len(train),len(test))

33119 9120


In [4]:
train.head()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,# Columns: time,FileName
0,42.0,0.0,18.5,0.5,12.0,0.0,0.0,bending1
1,42.0,0.0,18.0,0.0,11.33,0.94,250.0,bending1
2,42.75,0.43,16.75,1.79,18.25,0.43,500.0,bending1
3,42.5,0.5,16.75,0.83,19.0,1.22,750.0,bending1
4,43.0,0.82,16.25,0.83,18.0,0.0,1000.0,bending1


In [5]:
test.head()

,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,# Columns: time,FileName
0,39.25,0.43,22.75,0.43,33.75,1.3,0.0,bending1
1,39.25,0.43,23.0,0.0,33.0,0.0,250.0,bending1
2,39.25,0.43,23.25,0.43,33.0,0.0,500.0,bending1
3,39.5,0.5,23.0,0.71,33.0,0.0,750.0,bending1
4,39.5,0.5,24.0,0.0,33.0,0.0,1000.0,bending1


### (c) Feature Extraction
Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.

#### i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

Report ----------------- 

1. Simple statistical features
- Minimum
- Maximum
- Median
- Mean
- Standard Deviation
- Skewness
- Kurtosis
- Root mean square value (RMS)

2. Time serie analysis related features
- The 𝑑×𝑑−1 Cross-Correlations between each dimension 
- The 𝑑 Auto-Correlations
- Orders of the autoregressive (AR), integrated (I) and moving average (MA) part of an estimated ARIMA model
- Parameters of the AR part
- Parameters of the MA part

#### ii) Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly

In [6]:
colName = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
timeCol = ['min', 'max', 'mean', 'median', 'std', '25%', '75%']
newCol = ['instance','label']
for time in timeCol:
    for col in colName:
        newCol.append(time+'_'+col)
feature = pd.DataFrame(columns=newCol)

In [7]:
instance = 0
for dname in dirList:
    file = dirFile + dname 
    fileList = os.listdir(file+'/')
    for csv in fileList:
        temp_df = pd.read_csv(file+'/'+csv, skiprows=4)
        
        # for bending2-data4
        if temp_df.isnull().values.any():
            print("Different format in excel file : ", dname,'/',csv)
            for i in range(len(temp_df)):
                val = temp_df.loc[i, '# Columns: time'].split()
                temp_df.loc[i, '# Columns: time'] = val[0]
                for icol, col in enumerate(colName):
                    temp_df.loc[i, col] = float(val[icol+1])
            print("Format Change Completed: ", dname,'/',csv)
    
        des = temp_df.drop(columns=['# Columns: time']).describe()    
        
        row = {}
        row['instance'] = instance + 1
        row['label'] = dname
        for time in timeCol:
            for col in colName:    
                nc = time + '_' + col
                if time == 'median': 
                    row[nc] = des[col]['50%']
                else:
                    row[nc] = des[col][time]
        
        feature = pd.concat([feature, pd.DataFrame.from_records([row])],ignore_index=True)
        instance += 1

feature = feature.set_index('instance')
del feature['label']
feature = feature.astype(float)
feature

Different format in excel file :  bending2 / dataset4.csv
Format Change Completed:  bending2 / dataset4.csv


/Users/heejipark/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,min_avg_rss12,min_var_rss12,min_avg_rss13,min_var_rss13,min_avg_rss23,min_var_rss23,max_avg_rss12,max_var_rss12,max_avg_rss13,max_var_rss13,...,25%_avg_rss13,25%_var_rss13,25%_avg_rss23,25%_var_rss23,75%_avg_rss12,75%_var_rss12,75%_avg_rss13,75%_var_rss13,75%_avg_rss23,75%_var_rss23
instance,,,,,,,,,,,,,,,,,,,,,
1,36.25,0.0,1.50,0.0,11.33,0.00,48.00,1.50,26.33,5.17,...,14.25,0.0000,20.5000,0.0000,44.67,0.5000,18.0000,0.9400,23.75,0.8300
2,37.00,0.0,5.75,0.0,8.00,0.00,48.00,1.58,27.00,10.03,...,13.00,0.4300,22.2500,0.4300,45.00,0.5000,18.2700,1.1200,24.00,0.8700
3,33.00,0.0,8.50,0.0,20.00,0.00,47.75,3.00,30.00,5.15,...,20.50,0.4300,30.4575,0.0000,45.00,1.1200,24.3725,1.3000,36.33,1.0000
4,33.00,0.0,3.00,0.0,23.67,0.00,45.75,2.83,28.25,6.42,...,16.50,0.4300,28.4575,0.0000,42.75,0.7100,22.0625,1.1200,31.25,0.5000
5,37.25,0.0,4.00,0.0,27.25,0.00,45.00,1.30,29.50,7.23,...,16.00,0.0000,33.0000,0.0000,42.00,0.5000,23.2500,1.1200,36.00,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,19.75,0.0,6.25,0.0,6.25,0.00,45.50,13.47,22.25,9.00,...,13.50,2.1700,13.5000,2.1575,38.00,5.9700,17.0000,4.3900,17.75,4.5650
85,19.25,0.0,6.00,0.0,4.67,0.43,44.00,13.86,22.75,9.10,...,13.67,2.0500,13.7300,2.1200,38.00,5.9000,17.3300,4.0375,17.75,4.3750
86,23.50,0.0,6.67,0.0,5.50,0.00,46.25,14.82,24.25,9.90,...,13.50,2.0575,13.7500,2.1700,38.25,5.9325,17.2500,4.1900,18.00,4.5000


#### iii-1. Estimate the standard deviation of each of the time-domain features you extracted from the data. 

In [8]:
features_std = np.std(feature)
features_std

min_avg_rss12       9.515445
min_var_rss12       0.000000
min_avg_rss13       2.939616
min_var_rss13       0.000000
min_avg_rss23       6.089107
min_var_rss23       0.045577
max_avg_rss12       4.369322
max_var_rss12       5.033882
max_avg_rss13       4.847358
max_var_rss13       2.171183
max_avg_rss23       5.708524
max_var_rss23       2.504568
mean_avg_rss12      5.305314
mean_var_rss12      1.565194
mean_avg_rss13      3.985540
mean_var_rss13      1.159470
mean_avg_rss23      5.643253
mean_var_rss23      1.148232
median_avg_rss12    5.409056
median_var_rss12    1.404197
median_avg_rss13    4.013397
median_var_rss13    1.139058
median_avg_rss23    5.780655
median_var_rss23    1.080284
std_avg_rss12       1.762056
std_var_rss12       0.879068
std_avg_rss13       0.941316
std_var_rss13       0.455631
std_avg_rss23       1.019058
std_var_rss23       0.514668
25%_avg_rss12       6.118526
25%_var_rss12       0.940994
25%_avg_rss13       4.196608
25%_var_rss13       0.838813
25%_avg_rss23 

#### iii-2. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature

In [9]:
bootstrap_df = pd.DataFrame(columns=['feature', 'standard deviation','low_interval', 'high_interval'])

for col in list(feature.columns):
    
    feature_sequence = (np.array(feature[col]),)

    # Calculate 90% bootstrapped confidence interval for median (np.std 사용)
    bootstrap_ci = bootstrap(feature_sequence, np.std, confidence_level=0.90, method='percentile')

    # View 90% boostrapped confidence interval
    low, high = bootstrap_ci.confidence_interval
    
    # Save the result into dictionary
    row = {}
    row['feature'] = col
    row['standard deviation'] = features_std.loc[col]
    row['low_interval'] = low
    row['high_interval'] = high
    
    bootstrap_df = pd.concat([bootstrap_df, pd.DataFrame.from_records([row])],ignore_index=True)
    
bootstrap_df

,feature,standard deviation,low_interval,high_interval
0,min_avg_rss12,9.515445,8.231014,10.704314
1,min_var_rss12,0.0,0.0,0.0
2,min_avg_rss13,2.939616,2.742814,3.092009
3,min_var_rss13,0.0,0.0,0.0
4,min_avg_rss23,6.089107,4.346863,7.474412
5,min_var_rss23,0.045577,0.0,0.078029
6,max_avg_rss12,4.369322,3.325712,5.259189
7,max_var_rss12,5.033882,4.595192,5.368396
8,max_avg_rss13,4.847358,4.156582,5.442332
9,max_var_rss13,2.171183,1.963721,2.343186


#### iv. Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

Mean, Median, Standard Deviation

- Mean: It is the average of all values in a set. Therefore, it helps to measure the central tendency of data.
- Median: This is the middle number in a set of numbers sorted by size. This will be a good measure to figure out the central tendency of data. (If it the variables is skewed or the data have many outliers, Median is better representation.)
- Standard Deviation: It describes how dispersed a set of data is. It compares each data point to the mean of all data points, and standard deviation returns a calculated value that describes whether the data points are in close proximity or whether they are spread out. (A low standard deviation means that most of the numbers are close to the mean (average) value. A high standard deviation means that the values are spread out over a wider range. Therefore, it will help us to know the overal range of the data.)


    - In the case of the minimum and the maximum values, it will not very helpful when the data have many outliers.

<hr>

### 2. ISLR 3.7.4

<img src= './../data/ISLR-3-7-4/Page1.jpg'>
<img src= './../data/ISLR-3-7-4/Page2.jpg'>

### Reference
- Time-series features : https://stats.stackexchange.com/questions/50807/features-for-time-series-classification 
- Time-series features(std) : https://www.w3schools.com/python/python_ml_standard_deviation.asp, https://www.investopedia.com/terms/s/standarddeviation.asp#:~:text=Alex%20Dos%20Diaz-,What%20Is%20Standard%20Deviation%3F,deviation%20relative%20to%20the%20mean.
- bootstrap1 : https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html
- bootstrap2 : https://www.statology.org/bootstrapping-in-python/